In [ ]:
import torch
import os
import sys
sys.path.append("/content")


In [ ]:
rebuild_dataset = False
visualize_sample = True

ZIP_PATH = "/content/drive/MyDrive/datasets/soccertrack.zip"
TARGET_DIR = "/content/drive/MyDrive/datasets/soccertrack"
PREP_PATH = "/content/drive/MyDrive/datasets/soccertrack_prep"

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
# Get RT-DETR
if not os.path.isdir("/content/RT-DETR"):
  %cd /content
  print("Cloning RT-DETR")
  !git clone https://github.com/lyuwenyu/RT-DETR.git
else:
  print("RT-DETR is already cloned.")

%cd RT-DETR/rtdetrv2_pytorch
!pip install --upgrade pip -q
!pip install -r requirements.txt -q
%cd ..

# Get pretrained weights <- starting training from here
if not os.path.isfile("/content/RT-DETR/pretrained/rtdetrv2_r18vd_120e_coco.pth"):
  !mkdir -p pretrained
  !wget -P pretrained https://github.com/lyuwenyu/storage/releases/download/v0.1/rtdetrv2_r18vd_120e_coco.pth

In [ ]:
# Copy and unzip data in GDrive
if not os.path.isdir(TARGET_DIR) and rebuild_dataset:
    print("Unzipping soccertrack dataset into Google Drive ...")
    !unzip -q "$ZIP_PATH" -d "$TARGET_DIR"
else:
    print("Soccertrack dataset already exists, skipping unzip.")

In [ ]:
# Dataset processing
if rebuild_dataset:
    !pip install pandas pillow tqdm opencv-python -q

    from dataset_preparation import (
        extract_frames_from_videos,
        convert_soccertrack_csvs_to_coco
    )

    frame_out_root = f"{PREP_PATH}/frames"
    out_root = f"{PREP_PATH}/coco"

    video_root = f"{TARGET_DIR}/top_view/videos"
    annotation_root = f"{TARGET_DIR}/top_view/annotations"

    train_split = 0.8
    frame_stride = 20

    os.makedirs(PREP_PATH, exist_ok=True)

    if not os.path.isdir(frame_out_root):
        print(f"Extracting frames from {video_root} → {frame_out_root}")
        extract_frames_from_videos(
            video_root=video_root,
            output_root=frame_out_root,
            frame_stride=frame_stride
        )

    if not os.path.isdir(out_root):
        print("Converting SoccerTrack CSVs to COCO format")
        convert_soccertrack_csvs_to_coco(
            annotation_root=annotation_root,
            image_root=frame_out_root,
            output_root=out_root,
            train_split=train_split,
            frame_stride=frame_stride
        )


In [ ]:
if visualize_sample:
    !pip install matplotlib -q

    from visualization import visualize_coco_sample

    coco_root = "/content/drive/MyDrive/datasets/soccertrack_prep/coco"

    visualize_coco_sample(
        coco_json_path=f"{coco_root}/annotations/train.json",
        image_root=f"{coco_root}/images/train"
    )

In [ ]:
# Generate the config file for the dataset
%%bash
cd /content/RT-DETR/rtdetrv2_pytorch
cat << 'EOF' > configs/dataset/soccertrack_coco.yml
task: detection

evaluator:
  type: CocoEvaluator
  iou_types: ['bbox']

num_classes: 1
remap_mscoco_category: False

train_dataloader:
  type: DataLoader
  dataset:
    type: CocoDetection
    img_folder: /content/drive/MyDrive/datasets/soccertrack_prep/coco/images/train
    ann_file: /content/drive/MyDrive/datasets/soccertrack_prep/coco/annotations/train.json
    return_masks: False
    transforms:
      type: Compose
      ops: ~
  shuffle: True
  num_workers: 2
  drop_last: True
  collate_fn:
    type: BatchImageCollateFunction


val_dataloader:
  type: DataLoader
  dataset:
    type: CocoDetection
    img_folder: /content/drive/MyDrive/datasets/soccertrack_prep/coco/images/val
    ann_file: /content/drive/MyDrive/datasets/soccertrack_prep/coco/annotations/val.json
    return_masks: False
    transforms:
      type: Compose
      ops: ~
  shuffle: False
  num_workers: 2
  drop_last: False
  collate_fn:
    type: BatchImageCollateFunction
EOF


In [ ]:
if not os.path.isfile("/content/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_r18vd_120e_soccertrack.yml"):
  !cp /content/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_r18vd_120e_coco.yml /content/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_r18vd_120e_soccertrack.yml

!bash -c "sed -i \
  -e \"s|dataset:.*|dataset: /content/RT-DETR/rtdetrv2_pytorch/configs/dataset/soccertrack_coco.yml|\" \
  -e \"s|num_classes:.*|num_classes: 1|\" \
  -e \"s|pretrained:.*|pretrained: /content/RT-DETR/pretrained/rtdetrv2_r18vd_120e_coco.pth|\" \
  /content/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_r18vd_120e_soccertrack.yml; \
sed -i \"s|epochs: 120|epochs: 50|\" \
  /content/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_r18vd_120e_soccertrack.yml"
!grep -E "dataset:|num_classes:|pretrained:" \
  /content/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_r18vd_120e_soccertrack.yml

In [ ]:
device = torch.cuda.is_available()
print(f"GPU device for training / inference: {device}")